# Нормалізація даних

In [248]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import warnings
warnings.simplefilter('ignore')

_Імпортуємо данні_

In [249]:
data  = pd.read_csv('communities.csv', delimiter=',', header=None)
data


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12,?,?,TempleTerracecity,10,0.01,0.40,0.10,0.87,0.12,...,0.01,0.28,0.05,?,?,?,?,0.00,?,0.09
1990,6,?,?,Seasidecity,10,0.05,0.96,0.46,0.28,0.83,...,0.02,0.37,0.20,?,?,?,?,0.00,?,0.45
1991,9,9,80070,Waterburytown,10,0.16,0.37,0.25,0.69,0.04,...,0.08,0.32,0.18,0.08,0.06,0.78,0,0.91,0.28,0.23
1992,25,17,72600,Walthamcity,10,0.08,0.51,0.06,0.87,0.22,...,0.03,0.38,0.33,0.02,0.02,0.79,0,0.22,0.18,0.19


In [250]:
data.shape

(1994, 128)

_Проводимо обробку данних, встановлюємо назви колонок, змінюємо пусті значення на None та видаляємо їх, визначаємо колонки які містять інформативне значення, тобто назву міста, їх індекс тощо_ 

In [251]:
info_city = data[data.columns[:4]].copy()
ratio  = data.iloc[ : , 5:128].copy()
info_city.columns = ['index_1','index_2','index_2','City']
ratio.columns = [f'Ratio_{i+1}' for i in range(len(ratio.columns))]
df = pd.concat([info_city, ratio], axis=1)
df.rename(columns={'Ratio_123': 'Target'}, inplace=True)
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)
df.head()


,index_1,index_2,index_2,City,Ratio_1,Ratio_2,Ratio_3,Ratio_4,Ratio_5,Ratio_6,...,Ratio_114,Ratio_115,Ratio_116,Ratio_117,Ratio_118,Ratio_119,Ratio_120,Ratio_121,Ratio_122,Target
16,36,1,1000,Albanycity,0.15,0.31,0.40,0.63,0.14,0.06,...,0.06,0.39,0.84,0.06,0.06,0.91,0.5,0.88,0.26,0.49
23,19,193,93926,SiouxCitycity,0.11,0.43,0.04,0.89,0.09,0.06,...,0.16,0.12,0.07,0.04,0.01,0.81,1,0.56,0.09,0.63
33,51,680,47672,Lynchburgcity,0.09,0.43,0.51,0.58,0.04,0.01,...,0.14,0.11,0.19,0.05,0.01,0.75,0,0.60,0.1,0.31
68,34,23,58200,PerthAmboycity,0.05,0.59,0.23,0.39,0.09,1.00,...,0.01,0.73,0.28,0,0.02,0.64,0,1.00,0.23,0.50
74,9,9,46520,Meridentown,0.08,0.39,0.08,0.85,0.04,0.25,...,0.07,0.21,0.04,0.02,0.01,0.7,1,0.44,0.11,0.14


_Визначаємо предиктори , шукому велечину, одразу розбиваючі на тренеровочну та тестові вибірки, та нормалізуємо їх_

In [252]:
X = df.iloc[ : , 5:127].values
y = df['Target'].values


In [253]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.7, random_state=1)

In [254]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Linear Regression

_Загружаємо данні в лінійну регресію та дивимося метрики_

In [255]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [256]:
y_pred = linreg.predict(X_test)


In [257]:
print(f"Linear Regression-Training set score: {linreg.score(X_train, y_train):.2f}")
print(f"Linear Regression-Test set score: {linreg.score(X_test, y_test):.2f}")

Linear Regression-Training set score: 1.00
Linear Regression-Test set score: 0.73


## L2

In [258]:
ridge = Ridge(alpha=0.7).fit(X_train, y_train)
print(f"Ridge Regression-Training set score: {ridge.score(X_train, y_train):.2f}")
print(f"Ridge Regression-Test set score: {ridge.score(X_test, y_test):.2f}")

Ridge Regression-Training set score: 1.00
Ridge Regression-Test set score: 0.75


## L1

In [275]:
lasso = Lasso(alpha=0.05).fit(X_train, y_train)
print(f"Lasso Regression-Training set score: {lasso.score(X_train, y_train):.2f}")
print(f"Lasso Regression-Test set score: {lasso.score(X_test, y_test):.2f}")

Lasso Regression-Training set score: 0.97
Lasso Regression-Test set score: 0.96


## L1+L2

In [262]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.02, l1_ratio=0.01).fit(X_train, y_train)
print(f"Elastic Net-Training set score: {elastic_net.score(X_train, y_train):.2f}")
print(f"Elastic Net-Test set score: {elastic_net.score(X_test, y_test):.2f}")

Elastic Net-Training set score: 1.00
Elastic Net-Test set score: 0.84
